`\copy (SELECT * FROM qf_attempts a JOIN qf_users u ON a.user_id = u.id) TO '/tmp/sqlrn_attempts.csv' CSV HEADER`

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df = pd.read_csv('./data/mtmba_sqlrn_attempts.csv', index_col='id')
del df['id.1']
del df['created_at.1']
del df['updated_at.1']
del df['remember_token']
df['query'] = df['query'].str.strip()

In [ ]:
sc = pd.plotting.scatter_matrix(df, alpha=0.3, figsize=(10, 10),
diagonal='hist', marker='o', grid=False)

In [ ]:
counts = df.groupby(['exercise_id', 'valid']).agg({'query': {'attempts_count': 'count'}, 'username': {'distinct_users': pd.Series.nunique}})
counts.columns = counts.columns.droplevel(0)
counts.reset_index(level=1, inplace=True)
counts

In [ ]:
df.head()

In [ ]:
mtmba_status = df.pivot_table('username', index='exercise_id', columns='status', aggfunc='count')

In [ ]:
mtmba_status

In [ ]:
mtmba_status.plot()

In [ ]:
counts.sort_values('attempts_count', ascending=False, inplace=True)
counts[counts.valid == 't'][:30].plot(kind='bar')

In [ ]:
pd.set_option('display.max_colwidth', -1)

In [ ]:
plt.figure(figsize=(18,8))
date_histogram = df.groupby(pd.to_datetime(df.created_at).map(lambda x: x.date())).size()
#plt.subplot(211)
fig = date_histogram.plot(kind='bar')
plt.axvspan(0, 6, color='gray', alpha=0.1)
plt.axvspan(14, 20, color='gray', alpha=0.1)
plt.axvspan(28, 34, color='gray', alpha=0.1)
plt.axvspan(42, 48, color='gray', alpha=0.1)
plt.axvspan(56, 62, color='gray', alpha=0.1)
plt.axvspan(70, 76, color='gray', alpha=0.1)
plt.axvline(63, color='red', linestyle='--', linewidth=3)
plt.axvline(80, color='red', linestyle=':', linewidth=3)
fig.set_title('Number of attempts per day')
fig.set_xlabel('day')
fig.set_ylabel('no. of attempts')
plt.legend(['exam', 'resit exam'])
plt.savefig("days.png", bbox_inches='tight', pad_inches=0.1)
#plt.subplot(212)
#(students_df.groupby(pd.to_datetime(students_df.created_at).map(lambda x: x.weekday())).size()).plot(kind='bar')

In [ ]:
plt.figure(figsize=(18,8))
date_histogram = df.groupby(pd.to_datetime(df.created_at).map(lambda x: x.weekday())).size()
fig = date_histogram.plot(kind='bar')
fig.set_title('Number of attempts per week day')
fig.set_xlabel('Weekday')
fig.set_xticklabels(('Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'))
fig.set_ylabel('no. of attempts')
plt.savefig("weekdays.png")

In [ ]:
plt.figure
plt.figure(figsize=(18,4))
date_histogram = df.groupby(pd.to_datetime(df.created_at).map(lambda x: x.strftime('%H'))).size()
fig = date_histogram.plot(kind='bar')
fig.set_title('Number of attempts per hour of a day')
fig.set_xlabel('Hour of day')
fig.set_ylabel('no. of attempts')
plt.savefig("hours.png")

In [ ]:
attempts_by_exercise_user = df.loc[(df.status == 'f')].groupby(['exercise_id', 'user_id']).count().add_suffix('_count').reset_index()
avg_attempts_by_exercise = attempts_by_exercise_user.groupby('exercise_id')['query_count'].mean()
#fig = pd.DataFrame.boxplot(avg_attempts_by_exercise)
fig = avg_attempts_by_exercise.plot(kind='box')
fig.set_title('Average number of false \n attempts over all exercises')
fig.set_xlabel('')
fig.set_xticklabels((''))
#fig.set_xticklabels(('Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'))
fig.set_ylabel('no. of attempts')
plt.savefig("boxplot.svg")

In [ ]:
attempts_by_exercise_user = df.loc[(df.status == 'f')].groupby(['exercise_id', 'user_id']).count().add_suffix('_count').reset_index()
avg_attempts_by_exercise = attempts_by_exercise_user.groupby('exercise_id')['query_count'].quantile(0.99)
#fig = pd.DataFrame.boxplot(avg_attempts_by_exercise)
fig = avg_attempts_by_exercise.plot(kind='box')
fig.set_title('99 percentile of number of false \n attempts over all exercises')
fig.set_xlabel('')
fig.set_xticklabels((''))
fig.set_ylabel('no. of attempts')
plt.savefig("boxplot-99.svg")

In [ ]:
attempts_by_exercise_user = df.loc[(df.status == 'f')].groupby(['exercise_id', 'user_id']).count().add_suffix('_count').reset_index()
avg_attempts_by_exercise = attempts_by_exercise_user.groupby('user_id')['query_count'].mean()
#fig = pd.DataFrame.boxplot(avg_attempts_by_exercise)
plt.subplot(121)
plt.subplots_adjust(wspace=1.1)
fig = avg_attempts_by_exercise.plot(kind='box')
fig.set_title('No. of false attempts over all \n exercises (average per student)', y=1.05)
fig.set_xlabel('')
fig.set_xticklabels((''))
#fig.set_xticklabels(('Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'))
fig.set_ylabel('no. of attempts')

plt.subplot(122)
attempts_by_exercise_user = df.loc[(df.status == 'f')].groupby(['exercise_id', 'user_id']).count().add_suffix('_count').reset_index()
avg_attempts_by_exercise = attempts_by_exercise_user.groupby('user_id')['query_count'].quantile(0.99)
#fig = pd.DataFrame.boxplot(avg_attempts_by_exercise)
fig = avg_attempts_by_exercise.plot(kind='box')
fig.set_title('No. of false attempts over all  \n exercises (99 percentile per student)', y=1.05)
fig.set_xlabel('')
fig.set_xticklabels((''))
fig.set_ylabel('no. of attempts')

plt.savefig("boxplot-all-users.svg", bbox_inches='tight', pad_inches=0.1)

In [ ]:
import datetime
students_from_test = students_df.loc[pd.to_datetime(students_df.created_at).map(lambda x: x.date()) == datetime.date(2016, 6, 6,)]['user_id'].values

In [ ]:
tested_students = students_df[students_df.user_id.isin(students_from_test)]
max_no_of_exercises = tested_students[(tested_students.created_at < '2016-06-06')].exercise_id.nunique()
max_no_of_exam_ex = tested_students[(pd.to_datetime(students_df.created_at).map(lambda x: x) > datetime.datetime(2016, 6, 6, 9)) & 
                                    (pd.to_datetime(students_df.created_at).map(lambda x: x) < datetime.datetime(2016, 6, 6, 13))].exercise_id.nunique()
max_no_of_exam_ex
before_exam_counts = tested_students[(pd.to_datetime(students_df.created_at).map(lambda x: x) < datetime.datetime(2016, 6, 6, 9)) & 
                (tested_students.status == 't')].groupby(['user_id', 'exercise_id']).count().reset_index().groupby('user_id').count().ix[:,'exercise_id'].rename('before_exam_counts')
exam_counts = tested_students[(pd.to_datetime(students_df.created_at).map(lambda x: x) > datetime.datetime(2016, 6, 6, 9)) & 
                                    (pd.to_datetime(students_df.created_at).map(lambda x: x) < datetime.datetime(2016, 6, 6, 13)) & 
                (tested_students.status == 't')].groupby(['user_id', 'exercise_id']).count().reset_index().groupby('user_id').count().ix[:,'exercise_id'].rename('exam_counts')
exam_counts
joined = before_exam_counts.to_frame().join(exam_counts.to_frame()).fillna(0)

In [ ]:
import matplotlib.ticker as mtick
joined['before_ratio'] = joined['before_exam_counts'].map(lambda x: 100 * x/max_no_of_exercises)
joined['after_ratio'] = joined['exam_counts'].map(lambda x: 100 * x/max_no_of_exam_ex)
#joined[['before_ratio', 'after_ratio']]
plt.scatter(joined['before_ratio'].values, joined['after_ratio'].values)
fig = plt.gca()
fig.set_title('Ratio of solved exercises prior the exam vs \nratio of solved exercises on the exam')
fig.set_xlabel('solved exercises prior the exam')
fig.set_ylabel('solved exercises on the exam')
fmt = '%.0f%%' # Format you want the ticks, e.g. '40%'
xticks = mtick.FormatStrFormatter(fmt)
fig.xaxis.set_major_formatter(xticks)
fig.yaxis.set_major_formatter(xticks)
plt.savefig("correlation.svg",bbox_inches='tight', pad_inches=0.1)



In [ ]:
import scipy as sc
import scipy.stats
sc.stats.spearmanr(joined['before_ratio'].values, joined['after_ratio'].values)


In [ ]:
students_df.loc[(students_df.user_id == 47) & (students_df.status == 't')]